In [1]:
from pathlib import Path
from itertools import combinations, permutations
from tqdm import tqdm
import os
import random
import biodivine_aeon as ba
import pickle
import math


MODELS_FOLDER = ".\\models"
SELECTED_MODELS_EXP_1_FOLDER = "\\selected_models_exp_1"
CARDINALITIES_DICT_PICKL_FILE = 'cardinalities.pickle'
MAX_COMBINATIONS_NUM = 3000
file_cache = {}
RANDOM_SEED = 10
aeon_files = [f for f in os.listdir(MODELS_FOLDER)]


def load_boolean_network(file):
    if file not in file_cache.keys():
        file_cache[file] = Path(os.path.join(MODELS_FOLDER, file)).read_text()
    return ba.BooleanNetwork.from_aeon(file_cache[file])


def get_model_colours(file, variable_combination):
    tmp_model = load_boolean_network(file)
    for v in variable_combination:
        # Remove variable's update function; make it unknown
        tmp_model.set_update_function(v, None)

    graph = ba.PerturbationGraph(tmp_model)
    # The total colour of perturbed model
    full_colours = graph.as_original().unit_colors().cardinality()
    # 2^#vars colours come from artificial perturbation parameters (is_var_pertubred)
    colours_from_variables = 2 ** len(graph.as_original().network().variables())
    # The remaining colours come from model's own uncertainty
    colours_from_params = full_colours / colours_from_variables
    return colours_from_params

In [58]:
cardinalities_dict = {}

for f in aeon_files:
    print(f)
    model = load_boolean_network(f)
    vars = model.variables()
    rg = model.graph()

    # Variables with 0 arity have low effect, variables over 4 could explode colours too much
    relevant_vars = [model.get_variable_name(v) for v in vars if 0 < len(rg.regulators(v)) <= 4]

    # Generate possible combinations of relevant variables
    all_combinations = [x for x in combinations(relevant_vars, 4)]

    # Make random determinisitic
    random.seed(RANDOM_SEED)

    # If there are too many combinations, select just a sample
    combinations_to_try = all_combinations if len(all_combinations) <= MAX_COMBINATIONS_NUM else random.sample(all_combinations, MAX_COMBINATIONS_NUM)

    # For every model, remember a list of colours cardinalities along with the variable combinations which cause them
    cardinalities_dict[f] = list(tqdm(((vc, get_model_colours(f, vc)) for vc in combinations_to_try), total=len(combinations_to_try)))

cardiac_witness.aeon


100%|██████████| 330/330 [00:02<00:00, 112.78it/s]


erbb_witness.aeon


100%|██████████| 2380/2380 [02:25<00:00, 16.36it/s] 


mapk_witness.aeon


100%|██████████| 3000/3000 [18:53<00:00,  2.65it/s]


myeloid_witness.aeon


100%|██████████| 330/330 [00:06<00:00, 52.08it/s] 


tumour_witness.aeon


100%|██████████| 330/330 [01:50<00:00,  2.98it/s]


In [59]:
# Preserve results of long-running computation

with open(CARDINALITIES_DICT_PICKL_FILE, 'wb') as handle:
    pickle.dump(cardinalities_dict, handle)

In [60]:
# Load results of long-running computation

with open(CARDINALITIES_DICT_PICKL_FILE, 'rb') as handle:
    cardinalities_dict = pickle.load(handle)

In [61]:
print([y for _x,y in cardinalities_dict['myeloid_witness.aeon']])

[21772800.0, 87091200.0, 406425600.0, 406425600.0, 87091200.0, 406425600.0, 87091200.0, 87091200.0, 1959552.0, 9144576.0, 9144576.0, 1959552.0, 9144576.0, 1959552.0, 1959552.0, 36578304.0, 36578304.0, 7838208.0, 36578304.0, 7838208.0, 7838208.0, 170698752.0, 36578304.0, 170698752.0, 36578304.0, 36578304.0, 36578304.0, 170698752.0, 36578304.0, 36578304.0, 36578304.0, 7838208.0, 7838208.0, 36578304.0, 36578304.0, 7838208.0, 21772800.0, 101606400.0, 101606400.0, 21772800.0, 101606400.0, 21772800.0, 21772800.0, 406425600.0, 406425600.0, 87091200.0, 406425600.0, 87091200.0, 87091200.0, 1896652800.0, 406425600.0, 1896652800.0, 406425600.0, 406425600.0, 406425600.0, 1896652800.0, 406425600.0, 406425600.0, 406425600.0, 87091200.0, 87091200.0, 406425600.0, 406425600.0, 87091200.0, 9144576.0, 9144576.0, 1959552.0, 9144576.0, 1959552.0, 1959552.0, 42674688.0, 9144576.0, 42674688.0, 9144576.0, 9144576.0, 9144576.0, 42674688.0, 9144576.0, 9144576.0, 9144576.0, 1959552.0, 1959552.0, 9144576.0, 91445

In [62]:
sets = []
for _key, value in cardinalities_dict.items():
    sets.append(set([y for _x,y in value]))

available_colours_cardinality = set.intersection(*sets)
print(available_colours_cardinality)

{4665600.0, 18662400.0, 419904.0}


In [63]:
for v in available_colours_cardinality:
    print(f'Colours from params {v:.1f} (2^{math.log2(v)})')
    for m in aeon_files:
        satisfactory_var_combinations = [vc for vc, cols in cardinalities_dict[m] if cols == v]
        print(m, len(satisfactory_var_combinations))

Colours from params 4665600.0 (2^22.153631194101663)
cardiac_witness.aeon 36
erbb_witness.aeon 250
mapk_witness.aeon 210
myeloid_witness.aeon 10
tumour_witness.aeon 24
Colours from params 18662400.0 (2^24.153631194101663)
cardiac_witness.aeon 3
erbb_witness.aeon 50
mapk_witness.aeon 29
myeloid_witness.aeon 10
tumour_witness.aeon 4
Colours from params 419904.0 (2^18.67970000576925)
cardiac_witness.aeon 4
erbb_witness.aeon 50
mapk_witness.aeon 70
myeloid_witness.aeon 10
tumour_witness.aeon 2


In [64]:
selected_colours_cardinality = min(available_colours_cardinality)

for m in aeon_files:
    satisfactory_var_combination = [vc for vc, cols in cardinalities_dict[m] if cols == selected_colours_cardinality].pop()
    # Generate BN again (we saved just the unknown var combinations)
    model = load_boolean_network(m)
    for v in satisfactory_var_combination:
        # Remove variable's update function; make it unknown
        model.set_update_function(v, None)

    with open(f".\\{SELECTED_MODELS_EXP_1_FOLDER}\\{m.split('_')[0]}_4unknown.aeon", "wb") as handle:
        handle.write(model.to_aeon().encode())

In [2]:
model = load_boolean_network(f"tumour_witness.aeon")
vars = model.variables()
rg = model.graph()
print(len(vars))

# Variables with 0 arity have low effect, variables over 4 could explode colours too much
relevant_vars = [model.get_variable_name(v) for v in vars if 0 < len(rg.regulators(v)) <= 5]

# Generate possible ordering of relevant variables
all_perms = [x for x in permutations(relevant_vars, 5)]

32


In [10]:
for var in vars:
    v = model.get_variable_name(var)
    print(f"{v} has {len(rg.regulators(v))} regulators and {len(rg.targets(v))} targets")

AKT1 has 8 regulators and 10 targets
AKT2 has 7 regulators and 8 targets
Apoptosis has 8 regulators and 0 targets
CDH1 has 6 regulators and 4 targets
CDH2 has 1 regulators and 7 targets
CTNNB1 has 8 regulators and 10 targets
CellCycleArrest has 6 regulators and 0 targets
DKK1 has 2 regulators and 1 targets
DNAdamage has 0 regulators and 3 targets
ECM has 0 regulators and 2 targets
EMT has 2 regulators and 1 targets
ERK has 5 regulators and 3 targets
GF has 3 regulators and 4 targets
Invasion has 3 regulators and 1 targets
Metastasis has 1 regulators and 0 targets
Migration has 8 regulators and 1 targets
NICD has 6 regulators and 12 targets
SMAD has 3 regulators and 3 targets
SNAI1 has 6 regulators and 7 targets
SNAI2 has 6 regulators and 5 targets
TGFbeta has 3 regulators and 3 targets
TWIST1 has 3 regulators and 7 targets
VIM has 2 regulators and 1 targets
ZEB1 has 6 regulators and 5 targets
ZEB2 has 6 regulators and 7 targets
miR200 has 8 regulators and 9 targets
miR203 has 4 regulat

In [30]:
print(len(all_perms))

17160


In [68]:
tumour_stg = ba.SymbolicAsyncGraph(model)
attractors = ba.find_attractors(tumour_stg)

In [32]:
len(attractors)

3

In [54]:
apoptosis_ix = model.find_variable('Apoptosis')
print(apoptosis_ix)

VariableId(2)


In [52]:
print(attractors[0].vertices().list_vertices()[0][2])
print(attractors[1].vertices().list_vertices()[0][2])
print(attractors[2].vertices().list_vertices()[0][2])

True
True
False


In [77]:
selected_att = attractors[2]
# Make random determinisitic
random.seed(RANDOM_SEED)

# If there are too many combinations, select just a sample
permutations_to_try = all_perms if len(all_perms) <= MAX_COMBINATIONS_NUM else random.sample(all_perms, MAX_COMBINATIONS_NUM)

In [81]:
print(selected_att.vertices().list_vertices())

[[False, True, False, False, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False]]


In [85]:
def get_colours_with_attractor(pert_graph, original_attractor):
    seed = pert_graph.vertex(original_attractor.vertices().list_vertices()[0])
    fwd = ba.reach_fwd(pert_graph.as_perturbed(), seed)
    bwd = ba.reach_bwd(pert_graph.as_perturbed(), seed)
    scc = fwd.intersect(bwd)
    not_attractor_colors = fwd.minus(scc).colors()
    attractor = scc.minus_colors(not_attractor_colors)
    return attractor.colors()

def get_att_colours(vc, orginal_attractor):
    print("a")
    tmp_model = load_boolean_network(f"tumour_witness.aeon")
    for v in vc:
        # Remove variable's update function; make it unknown
        tmp_model.set_update_function(v, None)

    pert_graph = ba.PerturbationGraph(tmp_model)
    # The total colour of perturbed model's attractor
    return get_colours_with_attractor(pert_graph, orginal_attractor).cardinality()

def get_permutation_sequence(perm, original_attractor):
    return [get_att_colours(perm[:i+1], original_attractor) for i in range(len(perm))]

In [ ]:
cardinalities = list(tqdm(((ps, get_permutation_sequence(ps, selected_att)) for ps in permutations_to_try), total=len(permutations_to_try)))

  0%|          | 1/3000 [00:19<16:18:43, 19.58s/it]

In [36]:
# import os
# os.environ["PATH"] += os.pathsep + 'C:\\Program Files (x86)\\windows_10_msbuild_Release_graphviz-5.0.1-win32\\Graphviz\\bin'